In [8]:
%run -i bestestimator_v2.py

### Multiclass

In [9]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train.csv", sep = ',')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target.csv", sep = ';')

In [10]:
clf = BestEstimator(type_esti = 'Classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            view_nan = True,
        scoring = 'accuracy',
       value = 0) 

Missing Values :

                      Total       %
WARRANTIES_PRICE      96603  96.603
SHIPPING_PRICE        67610  67.610
BUYER_BIRTHDAY_DATE    5836   5.836
SHIPPING_MODE           315   0.315
PRICECLUB_STATUS         57   0.057
SELLER_SCORE_AVERAGE      6   0.006
SELLER_SCORE_COUNT        6   0.006 

NaN data filled by 0 

Searching for the best regressor on 1000 data using accuracy loss... 


 Bagging: 0.4730 (+/- 0.0096)

 Gradient Boosting: 0.4681 (+/- 0.0153)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



 XGBoost: 0.4900 (+/- 0.0156)

 Random Forest: 0.4990 (+/- 0.0075)

 Decision Tree: 0.3650 (+/- 0.0101)

 Extra Tree: 0.4540 (+/- 0.0285)

 KNN: 0.4470 (+/- 0.0126)

 SVM: 0.4870 (+/- 0.0027)

 Searching for the best hyperparametres of Random Forest on 1000 data among : 

{'n_estimators': [10, 100, 300], 'max_depth': [5, 10, None], 'criterion': ['gini', 'entropy']} 

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   18.2s finished



 Finally, the best estimator is : Random Forest Classifier

 Using these hyperparametres : {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}

 With this accuracy score : 0.51


In [12]:
clf.Bagg_fit(Train, Target, n_estimators = [10,40], ID = 'ID', metric = 'accuracy', n = 300)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   30.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   30.5s finished



 Best hyperparametres : {'n_estimators': 10}

 Giving this accuracy score : 0.5133333333333333
BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


### Binary

In [5]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train1.csv", sep = ';')
target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target1.csv", sep = ';')

Target = pd.DataFrame()
Target['ID'] = target['ID']
Target['Target'] = [np.nan]*target.shape[0]


for i in range(target.shape[0]):
    if target['Target'][i] == 1:
        Target['Target'][i] ='+'
    else :
        Target['Target'][i] ='-'
        
        

bestestimator_v2.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            view_nan = False,
        scoring = 'roc_auc') 

NaN data filled by 0 

Searching for the best regressor on 1000 data using roc_auc loss... 


 Bagging: 0.7116 (+/- 0.0077)

 Gradient Boosting: 0.7215 (+/- 0.0355)

 XGBoost: 0.7121 (+/- 0.0225)

 Random Forest: 0.7269 (+/- 0.0401)

 Decision Tree: 0.6278 (+/- 0.0293)

 Extra Tree: 0.6797 (+/- 0.0246)

 KNN: 0.6367 (+/- 0.0594)

 SVM: 0.5040 (+/- 0.0054)

 Searching for the best hyperparametres of Random Forest on 1000 data among : 

{'n_estimators': [10, 100, 300], 'max_depth': [5, 10, None], 'criterion': ['gini', 'entropy']} 

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   23.2s finished



 Finally, the best estimator is : Random Forest classifier

 Using these hyperparametres : {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 300}

 With this roc_auc score : 0.7407524989135159


In [11]:
clf.Bagg_fit(Train, Target, n_estimators = [10,20,100], ID = 'ID', metric = 'roc_auc', n = 500)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  3.0min finished



 Best hyperparametres : {'n_estimators': 100}

 Giving this roc_auc score : 0.718


### Regression

In [12]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\input_training.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target_Engie.csv", sep = ';')

In [13]:
clf = BestEstimator(type_esti = 'regressor', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 500,
            n_grid = 500,
            view_nan = True,
        scoring = 'r2') 

Missing Values :

                                Total          %
Grid_voltage                   101322  16.411451
Grid_voltage_std               101322  16.411451
Grid_voltage_max               101322  16.411451
Grid_voltage_min               101322  16.411451
Gearbox_inlet_temperature        8064   1.306152
Generator_converter_speed        8064   1.306152
Generator_converter_speed_min    8064   1.306152
Generator_converter_speed_max    8064   1.306152
Generator_converter_speed_std    8064   1.306152
Gearbox_inlet_temperature_min    8064   1.306152
Gearbox_inlet_temperature_max    8064   1.306152
Gearbox_inlet_temperature_std    8064   1.306152
Absolute_wind_direction_c          72   0.011662
Nacelle_angle_c                    72   0.011662 

NaN data filled by 0 

Searching for the best regressor on 500 data using r2 loss... 


 Bagging: 0.9634 (+/- 0.0235)

 Gradient Boosting: 0.9721 (+/- 0.0141)

 XGBoost: 0.9666 (+/- 0.0235)

 Random Forest: 0.9708 (+/- 0.0170)

 Decision Tree: 0

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  2.5min finished



 Finally, the best estimator is : Gradient Boosting regressor

 Using these hyperparametres : {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 5, 'n_estimators': 600}

 With this r2 score : 0.9750078443220153


In [14]:
clf.Bagg_fit(Train, Target, n_estimators = [10,20], ID = 'ID', metric = 'r2', n = 500, type_esti = 'Regressor')

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   43.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   43.7s finished



 Best hyperparametres : {'n_estimators': 20}

 Giving this r2 score : 0.1628245956749507
